"The code is implemented using Tensorflow and it is changed it to fully use Keras"
"This will allown for saving weights now."


In [ ]:
import numpy as np 
import pandas as pd 

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("inputs"))

In [ ]:
import os
import time
import tensorflow as tf
import numpy as np
import glob
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

# to choose which GPU should be used
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["HIP_VISIBLE_DEVICES"]="0"


# this section will allow code to be run on a RTX GPU
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


import warnings
#warnings.filterwarnings("ignore")

In [ ]:
import tensorflow.compat.v1.keras as keras
from tensorflow.compat.v1.keras.layers import Input, Dense, Reshape, Flatten, Dropout, GaussianNoise
from tensorflow.compat.v1.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.compat.v1.keras.layers import LeakyReLU
from tensorflow.compat.v1.keras.layers import UpSampling2D, Conv2D , Conv2DTranspose
from tensorflow.compat.v1.keras.models import Sequential, Model
from tensorflow.compat.v1.keras.optimizers import Adam

In [ ]:
# Create Generator
def get_generator(z):
    # 8 x 8 x 1024
    input_layer = Input(z)
    hid = Dense(8*8*1024, activation='relu', name ="Dense")(input_layer)    
    hid = LeakyReLU(alpha=0.2)(hid)
    hid = Reshape((8, 8, 1024))(hid)
    hid = GaussianNoise(0.02)(hid)
    
    # 8x8x512 -> 16x16x256
    hid = Conv2DTranspose(512, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv1")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv1")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv_out")(hid)
    #hid = Activation("relu", name ="trans_conv_out")(hid)
    
    # 8x8x512 -> 16x16x256
    hid = Conv2DTranspose(512, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv2")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv2")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv2_out")(hid)
    #hid = Activation("relu", name ="trans_conv2_out")(hid)
    
    
    # 16x16x256 -> 32x32x128
    hid = Conv2DTranspose(256, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv3")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv3")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv3_out")(hid)
    #hid = Activation("relu", name ="trans_conv3_out")(hid)
    

    # 32x32x128 -> 64x64x64
    hid = Conv2DTranspose(128, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv4")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv4")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv4_out")(hid)
    #hid = Activation("relu", name ="trans_conv4_out")(hid)  
    
    # 32x32x128 -> 64x64x64
    hid = Conv2DTranspose(64, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv5")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv5")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv5_out")(hid)
    #hid = Activation("relu", name ="trans_conv5_out")(hid)
    
    # 32x32x128 -> 64x64x64
    hid = Conv2DTranspose(64, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="trans_conv6")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_trans_conv6")(hid)
    hid = LeakyReLU(alpha=0.2,name ="trans_conv6_out")(hid)
    #hid = Activation("relu", name ="trans_conv6_out")(hid)
    
    # 64x64x64 -> 64x64x3
    hid = Conv2DTranspose(3, kernel_size=[5,5],
                               strides=[2,2],
                               padding="same",
                               kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),name ="logits")(hid)
  
    out = Activation("tanh", name ="out")(hid)
    
    model = Model(inputs=input_layer, outputs=out)
    print("Generator:")
    model.summary()
  
    return model

In [ ]:
# Create Discriminator

def get_discriminator(input_shape):
    # 64x64x3 -> 32x32x32
    input_layer = Input(input_shape)
    hid = Conv2D(filters=32,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv1")(input_layer)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm1")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv1_out")(hid)
    hid = Dropout(0.2)(hid)
    
    # 32x32x32-> 16x16x64 
    hid = Conv2D(filters=64,
                        kernel_size=[5,5],
                        strides=[2,2],
                        padding="same",
                        kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                        name = "conv2")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm2")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv2_out")(hid) 
    hid = Dropout(0.2)(hid)
    
    # 16x16x64  -> 8x8x128  
    hid = Conv2D(filters=128,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv3")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm3")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv3_out")(hid)
    hid = Dropout(0.2)(hid)
    
    # 8x8x128 -> 8x8x256
    hid = Conv2D(filters=256,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv4")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm4")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv4_out")(hid)
    hid = Dropout(0.2)(hid)
    
    # 8x8x128 -> 8x8x256
    hid = Conv2D(filters=256,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv5")(hid)
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm5")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv5_out")(hid)
    

    # 8x8x256 -> 4x4x512
    hid = Conv2D(filters=512,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv6")(hid)  
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm6")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv6_out")(hid)
    
    # 8x8x256 -> 4x4x512
    hid = Conv2D(filters=1024,
                    kernel_size=[5,5],
                    strides=[2,2],
                    padding="same",
                    kernel_initializer= keras.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV),
                    name = "conv7")(hid)  
    hid = BatchNormalization(momentum=0.9, epsilon=EPSILON, name="batch_norm7")(hid)
    hid = LeakyReLU(alpha=0.2, name="conv7_out")(hid)
    
    
    hid = Flatten(name = "flatten")(hid)

    out = Dense(1, activation='sigmoid', name = "ligit")(hid)
    model = Model(inputs= input_layer, outputs=out)
    print("Discriminator:")
    model.summary()

    return model

In [ ]:
# Hyperparameters
IMAGE_SIZE = 1024
NOISE_SIZE = 100

LR_D = 0.00003
LR_G = 0.0003
BATCH_SIZE = 6
EPOCHS = 2000 # For better results increase this value
BETA1 = 0.9
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.0005

#runtime 
SAMPLES_TO_SHOW = 6
SAVE_WEIGHTS_INTERVAL = 2
SHOW_PROGRES_INTERVAL = 1
LOAD_WEIGHTS_EP = 134

In [ ]:
# Paths
INPUT_DIR = "rect"
INPUT_DATA_DIR = "inputs/" + INPUT_DIR + "/" # Path to the folder with input images. For more info check simspons_dataset.txt

WEIGHTS_PATH = "weights" # for saving weights   
if not os.path.exists(WEIGHTS_PATH):
    os.makedirs(WEIGHTS_PATH)
    
OUTPUT_DIR = './gan-' + INPUT_DIR + '-{date:%Y-%m-%d_%H_%M_%S}/'.format(date=datetime.datetime.now())
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
MODEL_NAME = OUTPUT_DIR + "gan-" + str(IMAGE_SIZE) + "-" + str(NOISE_SIZE) + "-model." + str(LOAD_WEIGHTS_EP) + ".h5"    

In [ ]:
#Discriminator
discriminator = get_discriminator((IMAGE_SIZE, IMAGE_SIZE,3))


#discriminator.trainable = False

discriminator.compile(loss='binary_crossentropy',optimizer=Adam(lr=LR_D, beta_1=BETA1),metrics=['accuracy'])
#discriminator.load_weights("weights/disc-512-weights." + str(LOAD_WEIGHTS_EP) + ".h5");
# For the combined model we will only train the generator
discriminator.trainable = False

#Generator
generator = get_generator((NOISE_SIZE,))

#GAN
# The discriminator takes generated images as input and determines validity
gan_input = Input(shape=(NOISE_SIZE,))
x = generator(gan_input)
gan_out = discriminator(x)
gan = Model(gan_input, gan_out)
gan.summary()

gan.compile(loss='binary_crossentropy',optimizer=Adam(lr=LR_G, beta_1=BETA1))
if (LOAD_WEIGHTS_EP != 0):
    gan.load_weights("weights/gan-1024-" + str(NOISE_SIZE) + "-weights." + str(LOAD_WEIGHTS_EP) + ".h5", by_name=True)

#gan.save("models/gan-1024-model." + str(LOAD_WEIGHTS_EP) + ".h5")

In [ ]:
def show_samples(sample_images, name, epoch):
    figure, axes = plt.subplots(1, len(sample_images), figsize = (IMAGE_SIZE/8, IMAGE_SIZE/8))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = sample_images[index]
        axis.imshow(image_array)
        image = Image.fromarray(image_array)
        image.save(name+"_"+str(epoch)+"_"+str(index)+".png") 
    plt.savefig(name+"_"+str(epoch)+".png", bbox_inches='tight', pad_inches=0)
    print("save images")
    plt.show()
    plt.close()

In [ ]:
def test(input_z, epoch):
    samples = generator.predict(input_z[:SAMPLES_TO_SHOW])
    sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    show_samples(sample_images, OUTPUT_DIR + "samples", epoch)

In [ ]:
def summarize_epoch(d_losses, g_losses , data_shape, epoch, duration, input_z):
    minibatch_size = int(data_shape[0]//BATCH_SIZE)
    print("Epoch {}/{}".format(epoch, EPOCHS),
          "\nDuration: {:.5f}".format(duration),
          "\nD Loss: {:.5f}".format(np.mean(d_losses[-minibatch_size:])),
          "\nG Loss: {:.5f}".format(np.mean(g_losses[-minibatch_size:])))
    fig, ax = plt.subplots()
    plt.plot(d_losses, label='Discriminator', alpha=0.6)
    plt.plot(g_losses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.savefig(OUTPUT_DIR + "losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()
    test(input_z, epoch)

In [ ]:
def get_batches(data):
    batches = []
    for i in range(int(data.shape[0]//BATCH_SIZE)):
        #debug - 
        #print("get_batches_1: " + str(i))
        batch = data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        augmented_images = []
        for img in batch:
            image = Image.fromarray(img)
            if random.choice([True, False]):
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            augmented_images.append(np.asarray(image, dtype=np.float32))
        batch = np.asarray(augmented_images, dtype=np.float32)
        normalized_batch = (batch / 127.5) - 1.0        
        batches.append(normalized_batch)
    return np.array(batches)

In [ ]:
#Load Images

print("Load Sample Images")
input_images = np.asarray([np.asarray(Image.open(file).resize((IMAGE_SIZE, IMAGE_SIZE))) for file in glob(INPUT_DATA_DIR + '*')])

print ("Input: " + str(input_images.shape))

np.random.shuffle(input_images)

print("done shuffeling")

sample_images = random.sample(list(input_images), SAMPLES_TO_SHOW)
print("get random samples")

show_samples(sample_images, OUTPUT_DIR + "inputs", 0)
print("show samples")

# Training
print("Training Starts!")

#warnings.filterwarnings("ignore")

d_losses = []
g_losses = []
cum_d_loss = 0
cum_g_loss = 0

#EPOCHS = 50
for epoch in range(EPOCHS):
    epoch += 1
    start_time = time.time()
    batch =0;
    
    for batch_images in get_batches(input_images):
        batch +=1;
        #print ("training batch " + str(batch))
        noise_data = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_SIZE))
        # We use same labels for generated images as in the real training batch
        generated_images = generator.predict(noise_data)
        
        noise_prop = 0.05 # Randomly flip 5% of targets
        real_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(real_labels)), size=int(noise_prop*len(real_labels)))
        real_labels[flipped_idx] = 1 - real_labels[flipped_idx]
        
        # Train discriminator on real data
        d_loss_real = discriminator.train_on_batch(batch_images, real_labels)


        # Prepare labels for generated data
        fake_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(fake_labels)), size=int(noise_prop*len(fake_labels)))
        fake_labels[flipped_idx] = 1 - fake_labels[flipped_idx]
        
        # Train discriminator on generated data
        d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        cum_d_loss += d_loss
        d_losses.append(d_loss[0])
        
        
        # Train generator
        noise_data = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_SIZE))
        g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
        cum_g_loss += g_loss
        g_losses.append(g_loss)
        
    if epoch > 0 and epoch % SAVE_WEIGHTS_INTERVAL == 0 :
        print("saving model")
        aepochs = epoch + LOAD_WEIGHTS_EP
        #discriminator.save_weights("weights/disc-1024-weights."  + str(aepochs) + ".h5")
        #generator.save_weights("weights/gen-1024-weights."  + str(aepochs) + ".h5")
        gan.save_weights(OUTPUT_DIR + "gan-1024-" + str(NOISE_SIZE) + "-weights." + str(aepochs) + ".h5")
        print("saving")
    
    print("end epoch: " + str(epoch))    
    print("time elapsed: {:.5f}".format(time.time()-start_time))
  
    # Plot the progress
    if epoch > 0 and epoch % SHOW_PROGRES_INTERVAL == 0 :
        summarize_epoch(d_losses, g_losses, input_images.shape, epoch, time.time()-start_time, noise_data)
    batch_images = []